## Generate region file

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# load source code
filepath = "../data/ptr1440fov2_fmin10uJy_sources.csv"
pslist = pd.read_csv(filepath,comment="#",sep=",")

In [3]:
pslist.keys()

Index(['type', 'redshift', 'x', 'y', 'flux', 'major', 'minor', 'pa', 'x_l1',
       'y_l1', 'flux_l1', 'major_l1', 'minor_l1', 'pa_l1', 'x_l2', 'y_l2',
       'flux_l2', 'major_l2', 'minor_l2', 'pa_l2', 'x_h1', 'y_h1', 'flux_h1',
       'x_h2', 'y_h2', 'flux_h2'],
      dtype='object')

In [4]:
psarray = np.array(pslist)
pstype = np.array(pslist["type"])
psparam = psarray[:,1:]

locid = [1,2,6,7,12,13,18,19,22,23]
semiid = [4,5,9,10,15,16]
psparam[:,locid] += 1
psparam[:,semiid] /= 2

In [5]:
psarray.shape

(122344, 26)

In [6]:
psparam.shape

(122344, 25)

In [7]:
psarray[4920,:]

array([  1.00000000e+00,   2.66369000e-01,   2.20000000e+01,
         5.85000000e+02,   3.93550000e-05,              nan,
                    nan,   2.20456000e+01,   5.83271000e+02,
         1.55668000e-03,   2.15110000e+00,   6.85500000e-01,
         1.71544000e+02,   2.14120000e+01,   5.87526000e+02,
         1.55668000e-03,   2.15110000e+00,   6.85500000e-01,
         1.71544000e+02,              nan,              nan,
                    nan,              nan,              nan,
                    nan,              nan])

In [8]:
psparam[4920,:]

array([  2.66369000e-01,   2.20000000e+01,   5.85000000e+02,
         3.93550000e-05,              nan,              nan,
         2.20456000e+01,   5.83271000e+02,   1.55668000e-03,
         2.15110000e+00,   6.85500000e-01,   1.71544000e+02,
         2.14120000e+01,   5.87526000e+02,   1.55668000e-03,
         2.15110000e+00,   6.85500000e-01,   1.71544000e+02,
                    nan,              nan,              nan,
                    nan,              nan,              nan,
                    nan])

In [9]:
def getRegion(pstype,psparam):
    """Generate ds9 style region list"""
    if pstype == 1 or pstype == 2:
        id_l1 = np.array([7,6,9,10,11])
        id_l2 = np.array([13,12,15,16,17])
        reg1 = "ellipse(" + ",".join(psparam[id_l1].astype(str)) + ")\n"
        reg2 = "ellipse(" + ",".join(psparam[id_l2].astype(str)) + ")\n"
        reg = reg1+reg2
    elif pstype == 3:
        idx = [2,1]
        reg = "circle(" + ",".join(psparam[idx].astype(str)) + ", 1.0)\n"
    elif pstype == 4 or pstype == 5:
        idx = [2,1,4,5,6]
        reg = "ellipse(" + ",".join(psparam[idx].astype(str)) + ")\n"
    else:
        reg = None
    
    return reg

In [10]:
reg = getRegion(pstype[0], psparam[0,:])
print(reg)

circle(295.0,161.0, 1.0)



In [11]:
savefold = "./pslist/"

In [12]:
def writeRegion(pstype,psparam,idx_type,savepath):
    """Write ps list to region files."""
    with open(savepath, 'w') as fp:
        # ds9 style header
        fp.write('# Region file format: DS9 version 4.1\n')
        fp.write('global color=red dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')
        fp.write('image\n')
        for i in idx_type:
            reg = getRegion(pstype[i],psparam[i,:])
            fp.write(reg)

In [13]:
# Regions
for i in range(5):
    idx_type = np.where(pstype==i+1)[0]
    # save
    savepath = os.path.join(savefold,
                            "ptr1440fov2_fmin10uJy_type{0}.reg".format(str(i+1)))
    writeRegion(pstype,psparam,idx_type,savepath)